In [2]:
"""
Home Win dataset

create dataset with home_win from GCP to GCP
"""

'\nHome Win dataset\n\ncreate dataset with home_win from GCP to GCP\n'

In [3]:
# create table Home win dataset

# Import the important packages
import pandas as pd
import numpy as np
import datetime as dt
import getpass

# import schedule table
# Connexion GBQ
import pandas_gbq
from google.oauth2 import service_account


def get_keys():
    '''
    This function will return the path to Google Keys for each user.
    '''
    if getpass.getuser()=='antoinetl':
        credential_keys = '/Users/antoinetl/Documents/code/Google Keys/My First Project-4938b2ab0dc6.json'
    elif getpass.getuser()=='philippejacques':
        credential_keys = '/Users/philippejacques/Desktop/Projet/HockeyPrediction/Google Keys/rational-world-288611-dbe647d5aaf9.json'
    return credential_keys

credential_keys = get_keys()


credentials = service_account.Credentials.from_service_account_file(
    credential_keys,
    )
pandas_gbq.context.credentials = credentials
# https://pandas-gbq.readthedocs.io/en/latest/intro.html

In [4]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "rational-world-288611"

sql = """
SELECT Date, 
        gameID, 
        gameDate,
        teams_home_team_id, teams_away_team_id,
        teams_home_score,
        teams_away_score
from `My_dataset.schedule_and_results`
where gameType = 'R' and status_abstractGameState = 'Final'
order by date
"""
df_schedule_raw = pandas_gbq.read_gbq(sql, project_id=project_id)

sql = """
SELECT * from `My_dataset.teams_calendar` 
order by date
"""

df_cal_teams = pandas_gbq.read_gbq(sql, project_id = project_id)

Downloading: 100%|██████████| 42194/42194 [00:09<00:00, 4256.26rows/s]


In [5]:
df_schedule = df_schedule_raw
# date transformation
df_cal_teams['date'] = pd.to_datetime(df_cal_teams['date']).dt.tz_localize(None)
df_schedule['Date'] = pd.to_datetime(df_schedule['Date']).dt.tz_localize(None)
 
# Create date lag
df_schedule['date_lag'] = df_schedule['Date'] - dt.timedelta(days=1)
df_schedule.head()

# Get tonight's result
df_schedule = pd.merge(df_schedule, df_cal_teams[['wins_last_1', 'loss_last_1', 'ot_last_1', 'date','team']], left_on=['Date', 'teams_home_team_id'], right_on = ['date','team'], suffixes=('', '_home'))
df_schedule = pd.merge(df_schedule, df_cal_teams[['wins_last_1', 'loss_last_1', 'ot_last_1', 'date','team']], left_on=['Date', 'teams_away_team_id'], right_on = ['date','team'], suffixes=('', '_away'))

df_schedule = df_schedule.rename(columns={"wins_last_1": "Home_win", "loss_last_1": "Home_loss", "ot_last_1": 'Home_ot', 'ot_last_1_away': 'Away_ot'})
df_schedule = df_schedule.drop(['wins_last_1_away', 'loss_last_1_away', 'Date', 'gameDate', 'date_away', 'team_away', 'team'], axis=1)
#df = pd.merge(df_schedule,df_cal_teams[['Key_Column','Target_Column']],on='Key_Column', how='left')

df_schedule.head()

,gameID,teams_home_team_id,teams_away_team_id,teams_home_score,teams_away_score,date_lag,Home_win,Home_loss,Home_ot,date,Away_ot
0,2008020001,14,3,1,2,2008-10-03,0.0,1.0,0.0,2008-10-04,0.0
1,2008020002,9,5,3,4,2008-10-03,0.0,0.0,1.0,2008-10-04,0.0
2,2008020003,3,14,2,1,2008-10-04,1.0,0.0,0.0,2008-10-05,0.0
3,2008020004,5,9,1,3,2008-10-04,0.0,1.0,0.0,2008-10-05,0.0
4,2008020006,21,6,4,5,2008-10-08,0.0,1.0,0.0,2008-10-09,0.0


In [18]:
#df = df_schedule.merge(df_cal_teams, left_on=['date_lag', 'teams_home_team_id'], right_on=['date','team'], how = 'left', suffixes=('_left', '_right'))
df = pd.merge(df_schedule, df_cal_teams,  how='left', left_on=['date_lag', 'teams_home_team_id'], right_on = ['date','team'], suffixes=('', '_home'))

In [19]:
games_df = pd.merge(df, df_cal_teams, left_on=['date_lag', 'teams_away_team_id'], right_on = ['date','team'], suffixes=("", '_away'))

In [20]:
pd.set_option('display.max_columns', None)
games_df.describe(include = 'all')

,gameID,teams_home_team_id,teams_away_team_id,teams_home_score,teams_away_score,date_lag,Home_win,Home_loss,Home_ot,date,Away_ot,team,season,date_home,games_played,tonights_game_id,last_game_id,wins,losses,ot,points,ppg,shots_for_avg,shots_against_avg,shots_blocked_avg,goals_for_avg,goals_against_avg,hits_avg,pim_avg,powerPlayGoals_for_avg,powerPlayGoals_against_avg,powerPlayOpportunities_avg,takeaways_avg,giveaways_avg,wins_last_1,loss_last_1,ot_last_1,points_last_1,ppg_last_1,wins_last_10,loss_last_10,ot_last_10,points_last_10,ppg_last_10,team_away,season_away,date_away,games_played_away,tonights_game_id_away,last_game_id_away,wins_away,losses_away,ot_away,points_away,ppg_away,shots_for_avg_away,shots_against_avg_away,shots_blocked_avg_away,goals_for_avg_away,goals_against_avg_away,hits_avg_away,pim_avg_away,powerPlayGoals_for_avg_away,powerPlayGoals_against_avg_away,powerPlayOpportunities_avg_away,takeaways_avg_away,giveaways_avg_away,wins_last_1_away,loss_last_1_away,ot_last_1_away,points_last_1_away,ppg_last_1_away,wins_last_10_away,loss_last_10_away,ot_last_10_away,points_last_10_away,ppg_last_10_away
count,9.221000e+03,9221.000000,9221.000000,9221.000000,9221.000000,9221,9221.000000,9221.000000,9221.000000,9221,9221.000000,9187.000000,9187,9187,9187.000000,9.187000e+03,9093,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9187.000000,9221.000000,9221,9221,9221.000000,9.221000e+03,9086,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000,9221.000000
unique,NaN,NaN,NaN,NaN,NaN,1327,NaN,NaN,NaN,1327,NaN,NaN,8,1323,NaN,NaN,6831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,1327,NaN,NaN,6826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,2012-04-06 00:00:00,NaN,NaN,NaN,2014-10-11 00:00:00,NaN,NaN,20142015,2009-10-23 00:00:00,NaN,NaN,2009020255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20112012,2012-04-06 00:00:00,NaN,NaN,2013020837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,15,NaN,NaN,1213,15,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1219,15,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,2008-10-04 00:00:00,NaN,NaN,NaN,2008-10-05 00:00:00,NaN,NaN,NaN,2008-10-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-10-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2016-04-09 00:00:00,NaN,NaN,NaN,2016-04-10 00:00:00,NaN,NaN,NaN,2016-04-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.011493e+09,16.551676,16.559809,2.914543,2.636590,NaN,0.546145,0.333912,0.119944,NaN,0.119835,16.552520,NaN,NaN,39.789921,2.011493e+09,NaN,19.872973,15.176989,4.739959,64.358877,1.614710,29.927873,29.961090,13.932

In [52]:
games_df.head()

,gameID,teams_home_team_id,teams_away_team_id,teams_home_score,teams_away_score,date_lag,Home_win,Home_loss,Home_ot,date,Away_ot,team,season,date_home,games_played,tonights_game_id,last_game_id,wins,losses,ot,points,ppg,shots_for_avg,shots_against_avg,shots_blocked_avg,goals_for_avg,goals_against_avg,hits_avg,pim_avg,powerPlayGoals_for_avg,powerPlayGoals_against_avg,powerPlayOpportunities_avg,takeaways_avg,giveaways_avg,wins_last_1,loss_last_1,ot_last_1,points_last_1,ppg_last_1,wins_last_10,loss_last_10,ot_last_10,points_last_10,ppg_last_10,team_away,season_away,date_away,games_played_away,tonights_game_id_away,last_game_id_away,wins_away,losses_away,ot_away,points_away,ppg_away,shots_for_avg_away,shots_against_avg_away,shots_blocked_avg_away,goals_for_avg_away,goals_against_avg_away,hits_avg_away,pim_avg_away,powerPlayGoals_for_avg_away,powerPlayGoals_against_avg_away,powerPlayOpportunities_avg_away,takeaways_avg_away,giveaways_avg_away,wins_last_1_away,loss_last_1_away,ot_last_1_away,points_last_1_away,ppg_last_1_away,wins_last_10_away,loss_last_10_away,ot_last_10_away,points_last_10_away,ppg_last_10_away
0,2008020003,3,14,2,1,2008-10-04,1.0,0.0,0.0,2008-10-05,0.0,3.0,20082009,2008-10-04,1.0,2.008020e+09,None,1.0,0.0,0.0,3.0,3.0,41.0,21.0,6.0,2.0,2.0,16.0,8.0,1.0,0.0,7.0,19.0,8.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,0.3,14,20082009,2008-10-04,1,2008020001,None,0,1,0.0,0.0,0.0,21.0,41.0,13.0,1.0,2.0,17.0,14.0,0.0,1.0,4.0,11.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2008020004,5,9,1,3,2008-10-04,0.0,1.0,0.0,2008-10-05,0.0,5.0,20082009,2008-10-04,1.0,2.008020e+09,None,1.0,0.0,0.0,3.0,3.0,30.0,35.0,11.0,4.0,4.0,23.0,12.0,0.0,1.0,7.0,11.0,15.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,0.3,9,20082009,2008-10-04,1,2008020002,None,0,0,1.0,1.0,1.0,35.0,30.0,21.0,3.0,4.0,39.0,14.0,1.0,0.0,6.0,10.0,15.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.1
2,2008020022,5,1,1,2,2008-10-10,0.0,0.0,1.0,2008-10-11,0.0,5.0,20082009,2008-10-10,2.0,2.008020e+09,2008020002,1.0,1.0,0.0,3.0,1.5,30.5,31.0,13.0,2.5,3.5,27.5,11.0,0.5,1.5,7.0,9.0,13.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.3,1,20082009,2008-10-10,1,2008020009,None,1,0,0.0,3.0,3.0,29.0,26.0,8.0,2.0,1.0,21.0,14.0,1.0,1.0,4.0,10.0,1.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,0.0,3.0,0.3
3,2008020024,18,25,3,1,2008-10-10,1.0,0.0,0.0,2008-10-11,0.0,18.0,20082009,2008-10-10,1.0,2.008020e+09,None,0.0,1.0,0.0,0.0,0.0,22.0,22.0,7.0,2.0,2.0,23.0,12.0,0.0,4.0,4.0,3.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25,20082009,2008-10-10,1,2008020015,None,0,0,1.0,1.0,1.0,31.0,27.0,8.0,4.0,5.0,28.0,13.0,1.0,0.0,7.0,11.0,22.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.1
4,2008020020,15,16,4,2,2008-10-10,1.0,0.0,0.0,2008-10-11,0.0,15.0,20082009,2008-10-10,1.0,2.008020e+09,None,0.0,1.0,0.0,0.0,0.0,43.0,31.0,9.0,4.0,4.0,20.0,33.0,2.0,3.0,7.0,6.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16,20082009,2008-10-10,1,2008020010,None,0,1,0.0,0.0,0.0,32.0,29.0,12.0,2.0,2.0,31.0,16.0,0.0,0.0,3.0,8.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [53]:
games_df_prep = games_df[games_df['games_played'] > 15]
games_df_prep['ppg_diff'] = games_df_prep['ppg'] - games_df_prep['ppg_away']
games_df_prep['ppg_last_10_diff'] = games_df_prep['ppg_last_10'] - games_df_prep['ppg_last_10_away']

<ipython-input-53-61ad6f5be208>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df_prep['ppg_diff'] = games_df_prep['ppg'] - games_df_prep['ppg_away']
<ipython-input-53-61ad6f5be208>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df_prep['ppg_last_10_diff'] = games_df_prep['ppg_last_10'] - games_df_prep['ppg_last_10_away']


In [55]:
home_win_df = games_df_prep[[
    'gameID',
    'date',
    'teams_home_team_id',
    'teams_away_team_id',
    'Home_win',
    'ppg',
    'ppg_away',
    'ppg_diff',
    'ppg_last_10',
    'ppg_last_10_away',
    'ppg_last_10_diff'
]]

In [56]:
home_win_df.head()

,gameID,date,teams_home_team_id,teams_away_team_id,Home_win,ppg,ppg_away,ppg_diff,ppg_last_10,ppg_last_10_away,ppg_last_10_diff
194,2008020212,2008-11-09,24,13,0.0,1.750000,1.000000,0.750000,2.5,1.0,1.5
199,2008020213,2008-11-10,3,22,0.0,2.000000,1.571429,0.428571,1.6,1.0,0.6
204,2008020218,2008-11-11,20,10,1.0,1.562500,1.466667,0.095833,1.8,1.7,0.1
205,2008020220,2008-11-11,28,18,0.0,2.437500,1.357143,1.080357,2.4,1.3,1.1
219,2008020235,2008-11-13,28,20,1.0,2.352941,1.647059,0.705882,2.2,1.8,0.4


In [57]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "rational-world-288611"

# TODO: Set table_id to the full destination table ID (including the dataset ID).
table_id = 'My_dataset.home_win_df'

pandas_gbq.to_gbq(home_win_df, table_id, project_id=project_id, if_exists='replace')

1it [00:03,  3.96s/it]
